# Tutorial 4: RiboNN Translation Efficiency Prediction with Encodon

This notebook demonstrates predicting translation efficiency using pretrained Encodon models.

## Overview
- **Task**: Predict translation efficiency from mRNA sequences
- **Dataset**: RiboNN dataset with human translation efficiency data (see [1])
- **Model**: Pretrained Encodon + Random Forest regressor

[1] Zheng, Dinghai, et al. "Predicting the translation efficiency of messenger RNA in mammalian cells." Nature biotechnology (2025): 1-14.

## 1. Import Libraries and Setup

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import torch
import polars as pl
from pathlib import Path
from typing import Dict, List, Tuple
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Add project paths
sys.path.append('..')

# Import Encodon modules
from src.inference.encodon import EncodonInference
from src.inference.task_types import TaskTypes
from src.data.metadata import MetadataFields

# Fix random seed
torch.manual_seed(42)
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Load Pretrained Encodon Model

In [ ]:
# Define checkpoint paths

ckpt_path = "./"
AVAILABLE_MODELS = [
    f"{ckpt_path}/NV-CodonFM-Encodon-80M-v1/NV-CodonFM-Encodon-80M-v1.safetensors",
    f"{ckpt_path}/NV-CodonFM-Encodon-600M-v1/NV-CodonFM-Encodon-600M-v1.safetensors",
    f"{ckpt_path}/NV-CodonFM-Encodon-1B-v1/NV-CodonFM-Encodon-1B-v1.safetensors",
    f"{ckpt_path}/NV-CodonFM-Encodon-Cdwt-1B-v1/NV-CodonFM-Encodon-Cdwt-1B-v1.safetensors"
]
checkpoint_path = AVAILABLE_MODELS[0]

model_loaded = False
if os.path.exists(checkpoint_path):
    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Create EncodonInference wrapper
        encodon_model = EncodonInference(
            model_path=checkpoint_path,
            task_type=TaskTypes.EMBEDDING_PREDICTION,
        )
        
        # Configure model
        encodon_model.configure_model()
        encodon_model.to(device)
        encodon_model.eval()
        
        print(f"✅ Model loaded from: {checkpoint_path}")
        print(f"Device: {device}")
        print(f"Parameters: {sum(p.numel() for p in encodon_model.model.parameters()):,}")
        
        model_loaded = True        
    except Exception as e:
        print(f"Failed to load {checkpoint_path}: {e}")

if not model_loaded:
    print("❌ Could not load any model. Please check checkpoint paths.")

## 3. Load Dataset

In [ ]:
import os
import urllib.request
from pathlib import Path

# Configurable dataset path
data_path = "/data/validation/processed/data_with_human_TE_cellline_all_NA_plain.csv"

# Source URL for the TE dataset
te_dataset_url = "https://raw.githubusercontent.com/CenikLab/TE_classic_ML/refs/heads/main/data/data_with_human_TE_cellline_all_NA_plain.csv"

# Ensure parent directory exists
Path(os.path.dirname(data_path)).mkdir(parents=True, exist_ok=True)

# Download if missing
if not os.path.exists(data_path):
    print(f"Downloading TE dataset to {data_path} ...")
    urllib.request.urlretrieve(te_dataset_url, data_path)
    print("Download complete.")
else:
    print(f"Found existing dataset at {data_path}.")



In [ ]:


data = pl.read_csv(data_path, separator='\t')
data = data.with_columns([
            pl.struct(['utr5_size', 'cds_size', 'tx_sequence']).map_elements(
                lambda row: row['tx_sequence'][row['utr5_size']:row['utr5_size'] + row['cds_size']] ,
                return_dtype=pl.Utf8
            ).alias('cds_sequence'),
            pl.struct(['utr5_size', 'tx_sequence']).map_elements(
                lambda row: row['tx_sequence'][:row['utr5_size']],
                return_dtype=pl.Utf8
            ).alias('utr5_sequence'),
            pl.struct(['utr5_size', 'cds_size', 'tx_sequence']).map_elements(
                lambda row: row['tx_sequence'][row['utr5_size'] + row['cds_size']:],
                return_dtype=pl.Utf8
            ).alias('utr3_sequence')
        ]).with_row_index('id')
output_path = data_path[:-4] + '.processed.csv'
data.write_csv(output_path)

In [ ]:
# Load RiboNN dataset
data_loaded = False
if os.path.exists(output_path):
    try:
        data = pl.read_csv(output_path)
        print(f"✅ Loaded {len(data)} sequences from: {output_path}")
        print(f"Shape: {data.shape}")
        print(f"Key columns: {[col for col in ['id', 'cds_sequence', 'mean_te', 'fold'] if col in data.columns]}")
        
        data_loaded = True
    except Exception as e:
        print(f"Failed to load {output_path}: {e}")

    # Show basic statistics
    te_stats = data.select([
        pl.col('mean_te').mean().alias('mean'),
        pl.col('mean_te').std().alias('std'),
        pl.col('mean_te').min().alias('min'),
        pl.col('mean_te').max().alias('max')
    ])
    print(f"\nTranslation Efficiency stats:")
    print(f"  Mean: {te_stats['mean'][0]:.4f}")
    print(f"  Range: [{te_stats['min'][0]:.4f}, {te_stats['max'][0]:.4f}]")
    data_loaded = True

## 4. Data Preprocessing

In [ ]:
demo_size = 500 # set to len(data) for full dataset
batch_size = 16

# Subsample data while maintaining split proportions using sklearn
if data_loaded and demo_size < len(data):
    print(f"=== SUBSAMPLING DATA ===")
    sample_fraction = demo_size / len(data)
    _, data = train_test_split(
        data, 
        test_size=sample_fraction,
        stratify=data['fold'],
        random_state=42
    )
else:
    print(f"Using full dataset: {len(data) if data_loaded else 0} samples")


In [ ]:
demo_size = 500 # set to len(data) for full dataset
batch_size = 16
if data_loaded and model_loaded:
    print("=== DATA PREPROCESSING ===")
    data = data.to_pandas()
    sequences = data['cds_sequence'].tolist()
    targets = data['mean_te'].values    
    # Use subset for demo
    sequences = sequences
    targets = targets
    
    print(f"Processing {demo_size} sequences (demo mode)")
    
    # Extract embeddings
    print("\nExtracting embeddings...")
    all_embeddings = []
    
    for i in tqdm(range(0, len(sequences), batch_size)):
        batch_seqs = sequences[i:i + batch_size]
        
        # Prepare batch
        batch_items = []
        for j, seq in enumerate(batch_seqs):
            seq = seq.upper().replace('U', 'T')
            tokens = encodon_model.tokenizer.tokenize(seq)
            input_ids = encodon_model.tokenizer.convert_tokens_to_ids(tokens)
            
            # Truncate if needed
            if len(input_ids) > encodon_model.model.hparams.max_position_embeddings - 2:  # Leave room for CLS/SEP
                input_ids = input_ids[:encodon_model.model.hparams.max_position_embeddings - 2]
            
            # Add special tokens
            input_ids = [encodon_model.tokenizer.cls_token_id] + input_ids + [encodon_model.tokenizer.sep_token_id]
            attention_mask = [1] * len(input_ids)
            
            batch_items.append({
                MetadataFields.INPUT_IDS: input_ids,
                MetadataFields.ATTENTION_MASK: attention_mask,
            })
        
        # Pad batch
        max_len = encodon_model.model.hparams.max_position_embeddings
        
        padded_input_ids = []
        padded_attention_masks = []
        batch_ids = []
        
        for item in batch_items:
            input_ids = item[MetadataFields.INPUT_IDS]
            attention_mask = item[MetadataFields.ATTENTION_MASK]
            
            # Pad
            pad_len = max_len - len(input_ids)
            input_ids.extend([encodon_model.tokenizer.pad_token_id] * pad_len)
            attention_mask.extend([0] * pad_len)
            
            padded_input_ids.append(input_ids)
            padded_attention_masks.append(attention_mask)
        
        # Create batch tensor
        batch = {
            MetadataFields.INPUT_IDS: torch.tensor(padded_input_ids, dtype=torch.long).to(encodon_model.device),
            MetadataFields.ATTENTION_MASK: torch.tensor(padded_attention_masks, dtype=torch.long).to(encodon_model.device),
        }
        
        # Extract embeddings
        output = encodon_model.extract_embeddings(batch)
        all_embeddings.append(output.embeddings)
    
    # Combine embeddings
    embeddings = np.vstack(all_embeddings)
    print(f"\n✅ Extracted embeddings: {embeddings.shape}")
    
else:
    print("❌ Skipping preprocessing - data or model not loaded")

## 5. Train Random Forest

In [ ]:
if 'embeddings' in locals() and 'targets' in locals():
    print("=== TRAINING RANDOM FOREST ===")
    results = {'r2_scores': [], 'pearson_scores': [], 'mse_scores': []}
    folds = data['fold'].unique()
    for fold in folds:
        train_idx = data[data['fold'] != fold].index
        test_idx = data[data['fold'] == fold].index
        X_train, X_test = embeddings[train_idx], embeddings[test_idx]
        y_train, y_test = targets[train_idx], targets[test_idx]
        
        # Train Random Forest
        rf = RandomForestRegressor(
            n_estimators=500,
            max_depth=15,
            min_samples_split=2,
            random_state=42,
            n_jobs=-1
        )
        
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        
        # Calculate metrics
        r2 = r2_score(y_test, y_pred)
        pearson_r, _ = pearsonr(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        
        results['r2_scores'].append(r2)
        results['pearson_scores'].append(pearson_r)
        results['mse_scores'].append(mse)
        
        print(f"Fold {fold}: R² = {r2:.4f}, r = {pearson_r:.4f}")
    
    # Summary statistics
    mean_r2 = np.mean(results['r2_scores'])
    mean_pearson = np.mean(results['pearson_scores'])
    mean_mse = np.mean(results['mse_scores'])
    
    print(f"\n=== CROSS-VALIDATION RESULTS ===")
    print(f"Mean R²: {mean_r2:.4f} ± {np.std(results['r2_scores']):.4f}")
    print(f"Mean Pearson r: {mean_pearson:.4f} ± {np.std(results['pearson_scores']):.4f}")
    print(f"Mean RMSE: {np.sqrt(mean_mse):.4f}")
    
    # Store final model trained on all data
    final_rf = RandomForestRegressor(n_estimators=1000, max_depth=5, random_state=42, n_jobs=-1)
    final_rf.fit(embeddings, targets)
    final_predictions = final_rf.predict(embeddings)
    
else:
    print("❌ Cannot train - missing embeddings or targets")

## 6. Plot Results

In [ ]:
if 'final_predictions' in locals():
    fig, axes = plt.subplots(1, 2, figsize=(15, 8))
    fig.suptitle('RiboNN Translation Efficiency Prediction Results', fontsize=16)
    
    
    # Cross-validation performance
    axes[0].plot(range(len(results['r2_scores'])), results['r2_scores'], 'o-', label='R²')
    axes[0].plot(range(len(results['pearson_scores'])), results['pearson_scores'], 's-', label='Pearson r')
    axes[0].set_xlabel('Fold')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Cross-Validation Performance')
    
    # Add mean performance lines
    mean_r2 = np.mean(results['r2_scores'])
    mean_pearson = np.mean(results['pearson_scores'])
    axes[0].axhline(mean_r2, color='blue', linestyle='--', alpha=0.7, label=f'Mean R² = {mean_r2:.3f}')
    axes[0].axhline(mean_pearson, color='orange', linestyle='--', alpha=0.7, label=f'Mean Pearson r = {mean_pearson:.3f}')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Target distribution
    axes[1].hist(targets, bins=30, alpha=0.7, edgecolor='black')
    axes[1].axvline(targets.mean(), color='red', linestyle='--', label=f'Mean = {targets.mean():.3f}')
    axes[1].set_xlabel('Translation Efficiency')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Target Distribution')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No results to plot")

## 7. Troubleshooting & Optimization Tips

### Common Issues and Solutions:

#### 1. Model Loading Issues
- **Problem**: Checkpoint not found
- **Solution**: Update checkpoint paths in section 2
- **Check**: Verify checkpoint files exist and are accessible

#### 2. Data Loading Issues
- **Problem**: Dataset not found
- **Solution**: Update data paths in section 3
- **Check**: Ensure CSV files have required columns (id, ref_seq, value)

#### 3. Memory Issues
- **Problem**: CUDA out of memory
- **Solution**: Reduce batch_size in preprocessing section
- **Alternative**: Use CPU by setting device='cpu'

#### 4. Performance Issues
- **Problem**: Low R² scores
- **Solutions**:
  - Try larger models (600M or 1B parameters)
  - Implement fine-tuning instead of just embeddings
  - Tune Random Forest hyperparameters
  - Check data quality and preprocessing


### Optimization Strategies:

#### 1. Model Architecture
- **80M model**: Fast, good for initial experiments
- **600M model**: Better performance, moderate cost
- **1B model**: Best performance, highest computational cost

#### 3. Hyperparameter Tuning
```python
# Try these Random Forest parameters:
rf_params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
```